In [ ]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.optimize import Bounds

import warnings
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, TypedDict

from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
)
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite, select_individual_cloud_by_id


warnings.filterwarnings("ignore")

default_colors = set_custom_rcParams()

In [ ]:
def double_ln_normal_distribution(
    t: np.ndarray,
    mu1: float,
    sigma1: float,
    scale_factor1: float,
    mu2: float,
    sigma2: float,
    scale_factor2: float,
) -> np.ndarray:

    result = np.zeros(t.size)

    for mu, sigma, scale_factor in zip(
        (mu1, mu2),
        (sigma1, sigma2),
        (scale_factor1, scale_factor2),
    ):
        sigtilda = np.log(sigma)
        mutilda = np.log(mu)

        norm = scale_factor / (np.sqrt(2 * np.pi) * sigtilda)
        exponent = -((np.log(t) - mutilda) ** 2) / (2 * sigtilda**2)

        dn_dlnr = norm * np.exp(exponent)  # eq.5.8 [lohmann intro 2 clouds]

        result += dn_dlnr

    return result


def cost_function(x, t, y, var_scale=0.01, var_minimal=1e-12, var_replace=None):
    y_is = double_ln_normal_distribution(t, *x)

    # also devide by the variance of the data
    var = 1
    var = np.abs(var_scale * y)
    # # var = np.abs(var_scale * t ** (-0.5))

    if var_replace is None:
        var_min = np.where(var > var_minimal, var, np.nan)
        var_replace = np.nanmin(var_min)
    var = np.where(var <= var_minimal, var, var_replace)
    return np.ravel((y_is - y) / np.sqrt(var))


rng = default_rng()


def gen_data(
    t: np.ndarray,
    mu1: float,
    sigma1: float,
    scale_factor1: float,
    mu2: float,
    sigma2: float,
    scale_factor2: float,
    noise=0.0,
    n_outliers=0,
    seed=None,
):
    rng = default_rng(seed)

    y = double_ln_normal_distribution(
        t=t,
        mu1=mu1,
        sigma1=sigma1,
        scale_factor1=scale_factor1,
        mu2=mu2,
        sigma2=sigma2,
        scale_factor2=scale_factor2,
    )
    error = noise * rng.standard_normal(t.size)
    outliers = rng.integers(0, t.size, n_outliers)
    error[outliers] = np.sqrt(t[outliers]) * error[outliers]

    return y + error

In [ ]:
class DoubleLnParams(TypedDict):
    mu1: float
    sigma1: float
    scale_factor1: float
    mu2: float
    sigma2: float
    scale_factor2: float


params = DoubleLnParams(
    mu1=1e-2,
    sigma1=2,
    scale_factor1=5,
    mu2=0.5e1,
    sigma2=3,
    scale_factor2=1,
)

t_min = 0.1
t_max = 10
n_points = 40
n_outliers = 5

t_train = np.logspace(-3, 2, n_points)
m_train = gen_data(
    t=t_train,
    noise=0.2,
    n_outliers=n_outliers,
    **params,
)


x0 = np.array([1e-1, 2.0, 1.0, 10.0, 2.0, 1.0])


bounds = Bounds(
    lb=[1e-10, 1e-10, -np.inf, 2e-2, 1e-10, -np.inf],
    ub=[5e-1, np.inf, np.inf, np.inf, np.inf, np.inf],
    keep_feasible=[True, True, True, False, True, True],
)


res_lsq = least_squares(cost_function, x0, bounds=bounds, args=(t_train, m_train))

res_soft_l1 = least_squares(
    cost_function, x0, loss="soft_l1", f_scale=0.1, bounds=bounds, args=(t_train, m_train)
)

res_log = least_squares(
    cost_function, x0, loss="cauchy", f_scale=0.1, bounds=bounds, args=(t_train, m_train)
)


t_test = np.logspace(-5, 2, n_points * 10)
m_true = gen_data(
    t=t_test,
    **params,
)

m_lsq = gen_data(t_test, *res_lsq.x)
m_soft_l1 = gen_data(t_test, *res_soft_l1.x)
m_log = gen_data(t_test, *res_log.x)

plt.plot(t_train, m_train, "o")
plt.plot(t_test, m_true, "k", linewidth=2, label="true", linestyle="--")
plt.plot(t_test, m_lsq, label="linear loss", linestyle=":")
plt.plot(t_test, m_soft_l1, label="soft_l1 loss", linestyle="--")
plt.plot(t_test, m_log, label="cauchy loss", linestyle="-.")
plt.xlabel("t")
plt.ylabel("y")
plt.legend()
plt.xscale("log")

# Applying the ``least_square`` method to the cloud composite dataset

In [ ]:
cloud_composite = xr.open_dataset(
    "/home/m/m301096/repositories/sdm-eurec4a/data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
identified_clouds = xr.open_dataset(
    "/home/m/m301096/repositories/sdm-eurec4a/data/observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
)

attrs = cloud_composite["radius"].attrs.copy()
attrs.update({"units": "µm"})
cloud_composite["radius"] = cloud_composite["radius"]
cloud_composite["radius_micro"] = 1e6 * cloud_composite["radius"]
cloud_composite["radius"].attrs = attrs

# cloud_composite = cloud_composite.sel(radius = slice(10, None))

identified_clouds = identified_clouds.where(
    (
        (identified_clouds.duration.dt.total_seconds() > 5)
        & (identified_clouds.alt < 1300)
        & (identified_clouds.alt > 500)
    ),
    drop=True,
)
print(len(identified_clouds["cloud_id"]))
cloud_composite = match_clouds_and_cloudcomposite(identified_clouds, cloud_composite)

103


In [ ]:
coarse_composite = cloud_composite.coarsen(radius=3).sum()
coarse_composite["diameter"] = 2 * coarse_composite["radius"]
# normalize the particle size distirbution
attrs = coarse_composite["particle_size_distribution"].attrs.copy()
attrs["units"] = "m^-3 m^-1"
attrs["long_name"] = "Particle size distribution"
attrs["comment"] = "Each bin gives the number of droplets per cubic meter of air per meter of radius"

coarse_composite["particle_size_distribution"] = (
    coarse_composite["particle_size_distribution"] / coarse_composite["bin_width"] / 2
)
coarse_composite["particle_size_distribution"].attrs = attrs

# normalize the mass size distribution
attrs_mass = coarse_composite["mass_size_distribution"].attrs.copy()
attrs_mass["units"] = "kg m^-3 m^-1"
attrs_mass["long_name"] = "Mass size distribution"
attrs_mass["comment"] = "Each bin gives the mass of droplets per cubic meter of air per meter of radius"

coarse_composite["mass_size_distribution"] = (
    coarse_composite["mass_size_distribution"] / coarse_composite["bin_width"] / 2
)
coarse_composite["mass_size_distribution"].attrs = attrs_mass

In [ ]:
radius_split = 50e-6  # 50 µm


coarse_composite = cloud_composite.sel(radius=slice(radius_split, None)).coarsen(radius=3).sum()

coarse_composite["diameter"] = 2 * coarse_composite["radius"]

coarse_composite = xr.merge(
    [
        coarse_composite,
        cloud_composite.sel(radius=slice(None, radius_split)),
    ]
)


# # normalize the particle size distirbution
# attrs = coarse_composite['particle_size_distribution'].attrs.copy()
# attrs['units'] = 'm^-3 m^-1'
# attrs['long_name'] = 'Particle size distribution'
# attrs['comment'] = 'Each bin gives the number of droplets per cubic meter of air per meter of radius'

# coarse_composite['particle_size_distribution'] = (
#     coarse_composite['particle_size_distribution'] / coarse_composite['bin_width'] / 2
# )
# coarse_composite['particle_size_distribution'].attrs = attrs

# # normalize the mass size distribution
# attrs_mass = coarse_composite['mass_size_distribution'].attrs.copy()
# attrs_mass['units'] = 'kg m^-3 m^-1'
# attrs_mass['long_name'] = 'Mass size distribution'
# attrs_mass['comment'] = 'Each bin gives the mass of droplets per cubic meter of air per meter of radius'

# coarse_composite['mass_size_distribution'] = (
#     coarse_composite['mass_size_distribution'] / coarse_composite['bin_width'] / 2
# )
# coarse_composite['mass_size_distribution'].attrs = attrs_mass

In [ ]:
cloud_composite

In [ ]:
dataset = cloud_composite


fig, axs = plt.subplots(nrows=2, figsize=(8, 6), sharex=True)

# plot 5 individual random clouds
np.random.seed(42)
cloud_ids = rng.choice(identified_clouds["cloud_id"], 3, replace=False)

for i, cloud_id in enumerate(cloud_ids):
    cloud = select_individual_cloud_by_id(identified_clouds, cloud_id)
    ds = match_clouds_and_cloudcomposite(cloud, dataset)
    m, v = mean_and_stderror_of_mean(ds["particle_size_distribution"], dims=("time",))
    axs[0].errorbar(
        x=m["radius"],
        xerr=0,
        y=m,
        yerr=2 * v,
        label=f"cloud {cloud_id}",
        color=default_colors[i],
        marker=".",
        linestyle="None",
    )
    m, v = mean_and_stderror_of_mean(ds["mass_size_distribution"], dims=("time",))
    axs[1].errorbar(
        x=m["radius"],
        xerr=0,
        y=m,
        yerr=2 * v,
        label=f"cloud {cloud_id}",
        color=default_colors[i],
        marker=".",
        linestyle="None",
    )
    print(f"{cloud_id} {ds['mass_size_distribution'].sum('radius').mean('time').values} LWC")


m, v = mean_and_stderror_of_mean(dataset["particle_size_distribution"], dims=("time",))
axs[0].plot(
    m.radius,
    m,
    label="mean",
    color="k",
    zorder=10,
)
axs[0].fill_between(
    m.radius,
    m - 2 * v,
    m + 2 * v,
    alpha=0.5,
    color="k",
    label="mean ± std error",
    zorder=10,
)

m, v = mean_and_stderror_of_mean(dataset["mass_size_distribution"], dims=("time",))
axs[1].plot(m.radius, m, label="mean", color="k", zorder=10)
axs[1].fill_between(
    m.radius,
    m - 2 * v,
    m + 2 * v,
    alpha=0.5,
    color="k",
    label="mean ± std error",
    zorder=10,
)

axs[0].set_yscale("log")
for _ax in axs:
    _ax.legend(loc="upper center")
    _ax.set_xscale("log")
axs[1].set_ylim(0, 1e-5)

axs[0].set_ylabel("particle size distribution")
axs[1].set_ylabel("mass size distribution")

57 7.872833008032434e-05 LWC
40 6.697070801544104e-05 LWC
194 1.4557802384083504e-05 LWC


Text(0, 0.5, 'mass size distribution')

In [ ]:
dataset = coarse_composite


fig, axs = plt.subplots(nrows=2, figsize=(8, 6), sharex=True)


for i, cloud_id in enumerate(cloud_ids):
    cloud = select_individual_cloud_by_id(identified_clouds, cloud_id)
    ds = match_clouds_and_cloudcomposite(cloud, dataset)
    m, v = mean_and_stderror_of_mean(ds["particle_size_distribution"], dims=("time",))
    axs[0].errorbar(
        x=m["radius"],
        xerr=0,
        y=m,
        yerr=2 * v,
        label=f"cloud {cloud_id}",
        color=default_colors[i],
        marker=".",
        linestyle="None",
    )
    m, v = mean_and_stderror_of_mean(ds["mass_size_distribution"], dims=("time",))
    axs[1].errorbar(
        x=m["radius"],
        xerr=0,
        y=m,
        yerr=2 * v,
        label=f"cloud {cloud_id}",
        color=default_colors[i],
        marker=".",
        linestyle="None",
    )
    print(f"{cloud_id} {ds['mass_size_distribution'].sum('radius').mean('time').values} LWC")


m, v = mean_and_stderror_of_mean(dataset["particle_size_distribution"], dims=("time",))
axs[0].plot(
    m.radius,
    m,
    label="mean",
    color="k",
    zorder=10,
)
axs[0].fill_between(
    m.radius,
    m - 2 * v,
    m + 2 * v,
    alpha=0.5,
    color="k",
    label="mean ± std error",
    zorder=10,
)

m, v = mean_and_stderror_of_mean(dataset["mass_size_distribution"], dims=("time",))
axs[1].plot(m.radius, m, label="mean", color="k", zorder=10)
axs[1].fill_between(
    m.radius,
    m - 2 * v,
    m + 2 * v,
    alpha=0.5,
    color="k",
    label="mean ± std error",
    zorder=10,
)

axs[0].set_yscale("log")
for _ax in axs:
    _ax.legend(loc="upper center")
    _ax.set_xscale("log")
# axs[1].set_ylim(0, 1e-5)

axs[0].set_ylabel("particle size distribution")
axs[1].set_ylabel("mass size distribution")

57 7.872833008032434e-05 LWC
40 6.697070801544103e-05 LWC
194 1.4557802384083506e-05 LWC


Text(0, 0.5, 'mass size distribution')

In [ ]:
coarse_composite["radius2D"] = coarse_composite["radius"].expand_dims(time=coarse_composite["time"])
coarse_composite = coarse_composite.transpose("radius", ...)

In [ ]:
# chose random time step

# np.random.seed(42)
train_data = match_clouds_and_cloudcomposite(
    identified_clouds.isel(time=np.random.random_integers(0, len(identified_clouds.time) - 1)),
    coarse_composite,
)

t_train = train_data["radius2D"]  # .mean('time')
y_train = train_data["particle_size_distribution"]  # .mean('time')

In [ ]:
def double_ln_normal_distribution(
    t: np.ndarray,
    mu1: float,
    sigma1: float,
    scale_factor1: float,
    mu2: float,
    sigma2: float,
    scale_factor2: float,
) -> np.ndarray:

    result = np.zeros_like(t, dtype=float)

    for mu, sigma, scale_factor in zip(
        (mu1, mu2),
        (sigma1, sigma2),
        (scale_factor1, scale_factor2),
    ):
        sigtilda = np.log(sigma)
        mutilda = np.log(mu)

        norm = scale_factor / (np.sqrt(2 * np.pi) * sigtilda)
        exponent = -((np.log(t) - mutilda) ** 2) / (2 * sigtilda**2)

        dn_dlnr = norm * np.exp(exponent)  # eq.5.8 [lohmann intro 2 clouds]

        result += dn_dlnr

    return result


def fun_var(x, t, y):
    y_is = double_ln_normal_distribution(t, x[0], x[1], x[2], x[3], x[4], x[5])

    # also devide by the variance of the data
    var = 0.01 * y
    var = np.where(var != 0, var, 1e12)
    return (np.abs(y_is - y), 1 / np.sqrt(var))


def fun(x, t, y):
    y_is = double_ln_normal_distribution(t, x[0], x[1], x[2], x[3], x[4], x[5])

    # also devide by the variance of the data
    var = 0.01 * y
    var = np.where(var != 0, var, 1e30)
    return np.ravel((y_is - y) / np.sqrt(var))


def fun_no(x, t, y):
    y_is = double_ln_normal_distribution(t, x[0], x[1], x[2], x[3], x[4], x[5])

    # also devide by the variance of the data
    var = 1
    # var = np.where(var != 0, var, 1e30)
    return np.ravel((y_is - y) / np.sqrt(var))

In [ ]:
t, y = t_train.mean("time"), y_train.mean("time")
res, var = fun_var(x0, t, y)
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 5), sharex=True)
ax0, ax1, ax2 = axs
ax0.plot(t, y, marker="o", color=default_colors[0])
ax0.plot(t, res, marker="o", color=default_colors[1])
ax1.plot(t, var, marker="o", color=default_colors[3])
ax2.plot(t, res / y, marker="o", color=default_colors[2])
ax0.set_xscale("log")
ax0.set_yscale("log")
ax1.set_yscale("log")
# ax1.set_yscale('log')

In [ ]:
# np.random.seed(42)
train_data = match_clouds_and_cloudcomposite(
    identified_clouds.isel(time=np.random.random_integers(0, len(identified_clouds.time) - 1)),
    coarse_composite,
)

t_train = train_data["radius2D"]
y_train = train_data["particle_size_distribution"]
m_train = train_data["mass_size_distribution"]
w_train = train_data["bin_width"]

x0 = np.array([3e-6, 2, 1e10, 200e-6, 2, 1e6])
bounds = Bounds(
    lb=[1e-6, 1.1, 1e7, 50e-6, 1.1, 1e0],
    ub=[1e-5, 3.0, 1e13, 0.5e-3, 3.0, 1e8],
    # keep_feasible = [True, True, True, False, True, True]
)
res_lsq = least_squares(fun, x0, bounds=bounds, args=(t_train.mean("time"), y_train.mean("time")))
res_soft_l1 = least_squares(
    fun,
    x0,
    loss="soft_l1",
    f_scale=0.1,
    bounds=bounds,
    args=(t_train.mean("time"), y_train.mean("time")),
)
res_log = least_squares(
    fun, x0, loss="cauchy", f_scale=0.1, bounds=bounds, args=(t_train.mean("time"), y_train.mean("time"))
)
t_test = np.logspace(-6, -2.5, 1000)


y_lsq = gen_data(t_test, *res_lsq.x)
y_soft_l1 = gen_data(t_test, *res_soft_l1.x)
y_log = gen_data(t_test, *res_log.x)
y_init = gen_data(t_test, *x0)

# create mass size distirbution
m_lsq = y_lsq * t_test**3
m_soft_l1 = y_soft_l1 * t_test**3
m_log = y_log * t_test**3
m_init = y_init * t_test**3

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 5), sharex=True)
ax0, ax1 = axs

m_train = y_train * t_train**3

ax0.scatter(t_train, y_train, marker="o", color="grey")
ax0.scatter(t_train.mean("time"), y_train.mean("time"), marker="o", color="k")
ax0.set_xscale("log")
ax0.set_yscale("log")

ax1.scatter(t_train, m_train, marker="o", color="grey")
ax1.scatter(t_train.mean("time"), m_train.mean("time"), marker="o", color="k")
ax1.set_xscale("log")
# ax1.set_yscale('log')

for y, m in zip(
    (
        y_lsq,
        y_soft_l1,
    ),
    (
        m_lsq,
        m_soft_l1,
    ),
):
    ax0.plot(t_test, y, label="y")
    ax1.plot(t_test, m, label="m")


plt.xlabel("t")
plt.ylabel("y")
plt.legend()

w_test = t_test * 0
w_test[1:] = t_test[1:] - t_test[:-1]

# print(f"{(m_init * w_test).sum():.2e}')
print(f"{1e8 * (m_lsq * w_test).sum():.2f}")
print(f"{1e8 * (m_soft_l1 * w_test).sum():.2f}")
# print(f1e6 * '{(m_log * w_test).sum():.2f}')
print(f"{1e8 * (m_train * w_train).mean('time').sum(dim = 'radius').values:.2f} TRUTH")
# print(f"{(m_train * w_train).mean('time').sum().values:.2e}')

2.02
6.61
2.27 TRUTH


In [ ]:
# np.random.seed(42)
train_data = match_clouds_and_cloudcomposite(
    identified_clouds.isel(time=np.random.random_integers(0, len(identified_clouds.time) - 1)),
    coarse_composite,
)

t_train = train_data["radius2D"]  # .mean('time')
m_train = train_data["mass_size_distribution"]  # .mean('time')
w_train = train_data["bin_width"]
y_train = m_train * t_train ** (-3)

x0 = np.array([3e-6, 2, 1e-1, 300e-6, 2, 1e0])
bounds = Bounds(
    lb=[1e-6, 1.1, 1e-3, 50e-6, 1.1, 1e-3],
    ub=[1e-5, 4.0, 1e2, 0.5e-3, 3.0, 1e1],
    # keep_feasible = [True, True, True, False, True, True]
)
res_lsq = least_squares(fun, x0, bounds=bounds, args=(t_train.mean("time"), m_train.mean("time")))
res_soft_l1 = least_squares(
    fun,
    x0,
    loss="soft_l1",
    f_scale=0.1,
    bounds=bounds,
    args=(t_train.mean("time"), m_train.mean("time")),
)
res_log = least_squares(
    fun, x0, loss="cauchy", f_scale=0.1, bounds=bounds, args=(t_train.mean("time"), m_train.mean("time"))
)
t_test = np.logspace(-6, -2.5, 1000)


m_lsq = gen_data(t_test, *res_lsq.x)
m_soft_l1 = gen_data(t_test, *res_soft_l1.x)
m_log = gen_data(t_test, *res_log.x)
m_init = gen_data(t_test, *x0)

# create mass size distirbution
y_lsq = m_lsq * t_test ** (-3)
y_soft_l1 = m_soft_l1 * t_test ** (-3)
y_log = m_log * t_test ** (-3)
y_init = m_init * t_test ** (-3)

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 5), sharex=True)
ax0, ax1 = axs

m_train = y_train * t_train**3

ax0.scatter(t_train, y_train, marker="o", color="grey")
ax0.scatter(t_train.mean("time"), y_train.mean("time"), marker="o", color="k")
ax0.set_xscale("log")
ax0.set_yscale("log")

ax1.scatter(t_train, m_train, marker="o", color="grey")
ax1.scatter(t_train.mean("time"), m_train.mean("time"), marker="o", color="k")
ax1.set_xscale("log")
# ax1.set_yscale('log')

for y, m in zip(
    (y_lsq, y_soft_l1),
    (m_lsq, m_soft_l1),
):
    ax0.plot(t_test, y, label="y")
    ax1.plot(t_test, m, label="m")


plt.xlabel("t")
plt.ylabel("y")
plt.legend()

w_test = t_test * 0
w_test[1:] = t_test[1:] - t_test[:-1]

# print(f"{1e6 * (m_init * w_test).sum():.2f}')
print(f"{1e6 * (m_lsq * w_test).sum():.2f}")
print(f"{1e6 * (m_soft_l1 * w_test).sum():.2f}")
# print(f"{1e6 * (m_log * w_test).sum():.2f}')
# print(f1e6 * '{(m_train * w_train).sum(dim = 'radius').values:.2e} TRUTf")
print(f"{1e6 * (m_train * w_train).mean('time').sum().values:.2f}")

8.09
8.28
16.38


In [ ]:
lwc_org = []
lwc_lsq = []

for cloud_id in identified_clouds["cloud_id"]:
    # print('cloud_id', cloud_id.values)
    ds = select_individual_cloud_by_id(identified_clouds, cloud_id)
    train_data = match_clouds_and_cloudcomposite(ds, coarse_composite)

    t_train = train_data["radius2D"]  # .mean('time')
    m_train = train_data["mass_size_distribution"]  # .mean('time')
    w_train = train_data["bin_width"]
    y_train = m_train * t_train ** (-3)

    x0 = np.array([3e-6, 2, 1e-1, 300e-6, 2, 1e0])
    bounds = Bounds(
        lb=[1e-6, 1.1, 1e-3, 50e-6, 1.1, 1e-3],
        ub=[1e-5, 4.0, 1e2, 0.5e-3, 3.0, 1e1],
        # keep_feasible = [True, True, True, False, True, True]
    )
    res_lsq = least_squares(fun, x0, bounds=bounds, args=(t_train, m_train))

    t_test = np.logspace(-6, -2.5, 200)
    w_test = t_test * 0
    w_test[1:] = t_test[1:] - t_test[:-1]

    m_lsq = gen_data(t_test, *res_lsq.x)
    m_init = gen_data(t_test, *x0)

    # create mass size distirbution
    y_lsq = m_lsq * t_test ** (-3)
    y_init = m_init * t_test ** (-3)

    lwc_org.append(1e6 * (m_train * w_train).mean("time").sum().values)
    lwc_lsq.append(1e6 * (m_lsq * w_test).sum())

In [ ]:
# print(f"{1e6 * (m_init * w_test).sum():.2f}')
plt.scatter(lwc_org, lwc_lsq, color="k", marker="o")
plt.plot(
    [0, 500],
    [0, 500],
)
# print(f"{1e6 * (m_lsq * w_test).sum():.2f}')
# print(f"{1e6 * (m_train * w_train).mean('time').sum().values:.2f}')

In [ ]:
res_lsq = least_squares(fun_no, x0, bounds=bounds, args=(t_train, y_train))
res_soft_l1 = least_squares(
    fun_no,
    x0,
    loss="soft_l1",
    f_scale=0.1,
    bounds=bounds,
    args=(t_train, y_train),
)
res_log = least_squares(fun_no, x0, loss="cauchy", f_scale=0.1, bounds=bounds, args=(t_train, y_train))
t_test = np.logspace(-6, -2.9, 1000)


y_lsq = gen_data(t_test, *res_lsq.x)
y_soft_l1 = gen_data(t_test, *res_soft_l1.x)
y_log = gen_data(t_test, *res_log.x)
y_init = gen_data(t_test, *x0)

plt.scatter(t_train, y_train, marker="o", color="grey")
plt.scatter(t_train.mean("time"), y_train.mean("time"), marker="o", color="k")
plt.plot(t_test, y_lsq, label="linear loss")
plt.plot(t_test, y_soft_l1, label="soft_l1 loss")
plt.plot(t_test, y_log, label="cauchy loss")
plt.plot(t_test, y_init, label="innit")
plt.xlabel("t")
plt.ylabel("y")
plt.legend()
plt.xscale("log")
plt.yscale("log")
plt.ylim(1e-6, 1e1)

w_test = t_test * 0
w_test[1:] = t_test[1:] - t_test[:-1]

print(f"{(y_lsq * w_test).sum():.2e}")
print(f"{(y_train * w_train).mean('time').sum().values:.2e}")

3.71e-05
8.06e-05
